In [ ]:
import os
import numpy as np
from os import listdir
from matplotlib import pyplot as plt
from scipy.misc import imread, imresize, toimage, imsave
from sklearn.model_selection import train_test_split

In [ ]:
def get_img(data_path):
    img = imread(data_path)
    img = imresize(img, (256, 256))
    return img

In [ ]:
def save_img(img, name='mask.tif'):
    imsave(name, img.reshape(256, 256))

In [ ]:
def get_dataset(dataset_path='H:\\testovoe\\Ultrasound Nerve Segmentation\\raw\\train'):
    try:
        X = np.load('H:\\testovoe\\Ultrasound Nerve Segmentation\\raw\\npy_train_data\\X.npy')
        Y = np.load('H:\\testovoe\\Ultrasound Nerve Segmentation\\raw\\npy_train_data\\Y.npy')
    except:
        images = listdir(dataset_path)
        X = []
        Y = []
        for img in images:
            if 'mask' in img:
                continue
            img_path = dataset_path+'/'+img
            X.append(get_img(img_path).astype('float32').reshape(256, 256, 1)/255.)
            Y.append(get_img(img_path.replace('.', '_mask.')).astype('float32').reshape(256, 256, 1)/255.)
        X = np.array(X)
        Y = np.array(Y)
        if not os.path.exists('H:\\testovoe\\Ultrasound Nerve Segmentation\\raw\\npy_train_data\\'):
            os.makedirs('H:\\testovoe\\Ultrasound Nerve Segmentation\\raw\\npy_train_data\\')
        np.save('H:\\testovoe\\Ultrasound Nerve Segmentation\\raw\\npy_train_data\\X.npy', X)
        np.save('H:\\testovoe\\Ultrasound Nerve Segmentation\\raw\\npy_train_data\\Y.npy', Y)
    X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
    return X, X_test, Y, Y_test

In [ ]:
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
from keras.models import Model
from keras import backend as K
from keras.models import model_from_json
from keras.layers import Input, Conv2D, UpSampling2D, Activation, MaxPooling2D, Flatten, Dense, concatenate, Dropout


def get_model():

    inputs = Input(shape=(256, 256, 1))
    
    conv_block_1 = Conv2D(16, (3, 3), strides=(1, 1), padding='same')(inputs)
    conv_block_1 = Activation('relu')(conv_block_1)
    conv_block_1 = Conv2D(16, (3, 3), strides=(1, 1), padding='same')(conv_block_1)
    conv_block_1 = Activation('relu')(conv_block_1)
    pool_block_1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv_block_1)
    
    conv_block_2 = Conv2D(32, (3, 3), strides=(1, 1), padding='same')(pool_block_1)
    conv_block_2 = Activation('relu')(conv_block_2)
    conv_block_2 = Conv2D(32, (3, 3), strides=(1, 1), padding='same')(conv_block_2)
    conv_block_2 = Activation('relu')(conv_block_2)
    pool_block_2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv_block_2)
    
    conv_block_3 = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(pool_block_2)
    conv_block_3 = Activation('relu')(conv_block_3)
    conv_block_3 = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(conv_block_3)
    conv_block_3 = Activation('relu')(conv_block_3)
    pool_block_3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv_block_3)
    
    conv_block_4 = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(pool_block_3)
    conv_block_4 = Activation('relu')(conv_block_4)
    conv_block_4 = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(conv_block_4)
    conv_block_4 = Activation('relu')(conv_block_4)
    pool_block_4 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv_block_4)
    
    conv_block_5 = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(pool_block_4)
    conv_block_5 = Activation('relu')(conv_block_5)
    conv_block_5 = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(conv_block_5)
    conv_block_5 = Activation('relu')(conv_block_5)
    
    up_block_1 = UpSampling2D((2, 2))(conv_block_5)
    up_block_1 = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(up_block_1)
    
    merge_1 = concatenate([conv_block_4, up_block_1])
    
    conv_block_6 = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(merge_1)
    conv_block_6 = Activation('relu')(conv_block_6)
    conv_block_6 = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(conv_block_6)
    conv_block_6 = Activation('relu')(conv_block_6)
    
    up_block_2 = UpSampling2D((2, 2))(conv_block_6)
    up_block_2 = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(up_block_2)
    
    merge_2 = concatenate([conv_block_3, up_block_2])
    
    conv_block_7 = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(merge_2)
    conv_block_7 = Activation('relu')(conv_block_7)
    conv_block_7 = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(conv_block_7)
    conv_block_7 = Activation('relu')(conv_block_7)
    
    up_block_3 = UpSampling2D((2, 2))(conv_block_7)
    up_block_3 = Conv2D(32, (3, 3), strides=(1, 1), padding='same')(up_block_3)
    
    merge_3 = concatenate([conv_block_2, up_block_3])
    
    conv_block_8 = Conv2D(32, (3, 3), strides=(1, 1), padding='same')(merge_3)
    conv_block_8 = Activation('relu')(conv_block_8)
    conv_block_8 = Conv2D(32, (3, 3), strides=(1, 1), padding='same')(conv_block_8)
    conv_block_8 = Activation('relu')(conv_block_8)
    
    up_block_4 = UpSampling2D((2, 2))(conv_block_8)
    up_block_4 = Conv2D(16, (3, 3), strides=(1, 1), padding='same')(up_block_4)
    
    merge_4 = concatenate([conv_block_1, up_block_4])
    
    conv_block_9 = Conv2D(16, (3, 3), strides=(1, 1), padding='same')(merge_4)
    conv_block_9 = Activation('relu')(conv_block_9)
    conv_block_9 = Conv2D(16, (3, 3), strides=(1, 1), padding='same')(conv_block_9)
    conv_block_9 = Activation('relu')(conv_block_9)
    
    conv_block_10 = Conv2D(1, (1, 1), strides=(1, 1), padding='same')(conv_block_9)
    outputs = Activation('sigmoid')(conv_block_10)
    
    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='adadelta', loss=dice_coef_loss, metrics=[dice_coef])

    return model


def save_model(model):
    if not os.path.exists('H:\\testovoe\\Ultrasound Nerve Segmentation\\'):
        os.makedirs('H:\\testovoe\\Ultrasound Nerve Segmentation\\')
    model_json = model.to_json()
    with open("H:\\testovoe\\Ultrasound Nerve Segmentation\\model.json", "w") as model_file:
        model_file.write(model_json)
    model.save_weights("H:\\testovoe\\Ultrasound Nerve Segmentation\\weights.h5")
    print('Model and weights saved')
    return

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard

epochs = 25
batch_size = 256

In [ ]:
def train_model(model, X, X_test, Y, Y_test):
    if not os.path.exists('H:\\testovoe\\Ultrasound Nerve Segmentation\\Checkpoints\\'):
        os.makedirs('H:\\testovoe\\Ultrasound Nerve Segmentation\\Checkpoints\\')
    checkpoints = []
    checkpoints.append(ModelCheckpoint('H:\\testovoe\\Ultrasound Nerve Segmentation\\Checkpoints\\best_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1))
    checkpoints.append(TensorBoard(log_dir='H:\\testovoe\\Ultrasound Nerve Segmentation\\Checkpoints\\.\\logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None))

    model.fit(X, Y, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test), shuffle=True, callbacks=checkpoints)

    return model

In [ ]:
def main():
    X, X_test, Y, Y_test = get_dataset()
    model = get_model()
    model = train_model(model, X, X_test, Y, Y_test)
    save_model(model)
    return model

main()

In [ ]:
def predict(model, X):
    X = X.reshape(1, 256, 256, 1)
    Y = model.predict(X)*255.
    return Y